[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Hawksight-AI/semantica/blob/main/cookbook/advanced/08_Reasoning_and_Inference.ipynb)

# Reasoning and Inference

## Overview

Build knowledge graphs, define rules, perform forward/backward chaining, and generate explanations for AI reasoning using the **Semantica Reasoning Module**.


**Documentation**: [API Reference](https://semantica.readthedocs.io/reference/reasoning/)

## Installation

Install Semantica from PyPI:

```bash
pip install semantica
# Or with all optional dependencies:
pip install semantica[all]
```

## Workflow: Build KG → Define Rules → Forward/Backward Chaining → Generate Explanations


In [2]:
!pip install -qU semantica


In [3]:
from semantica.kg import GraphBuilder
from semantica.reasoning import InferenceEngine, RuleManager, ExplanationGenerator


## Step 1: Build Knowledge Graph


In [4]:
builder = GraphBuilder()

entities = [
    {"id": "alice", "type": "Person", "name": "Alice"},
    {"id": "bob", "type": "Person", "name": "Bob"},
    {"id": "charlie", "type": "Person", "name": "Charlie"},
    {"id": "sf", "type": "Location", "name": "San Francisco"},
    {"id": "california", "type": "Location", "name": "California"},
]

relationships = [
    {"source": "alice", "target": "bob", "type": "parent_of"},
    {"source": "bob", "target": "charlie", "type": "parent_of"},
    {"source": "sf", "target": "california", "type": "located_in"},
    {"source": "alice", "target": "sf", "type": "lives_in"},
]

knowledge_graph = builder.build([{"entities": entities, "relationships": relationships}])


Status,Action,Module,Submodule,File,Time
✅,Semantica is building,🧠 kg,GraphBuilder,-,0.31s
🔄,Semantica is building,🧠 kg,EntityResolver,-,2.16s
✅,Semantica is deduplicating,🔄 deduplication,DuplicateDetector,-,0.02s
✅,Semantica is deduplicating,🔄 deduplication,SimilarityCalculator,-,0.01s
✅,Semantica is deduplicating,🔄 deduplication,EntityMerger,-,0.05s
✅,Semantica is deduplicating,🔄 deduplication,MergeStrategyManager,-,0.01s
✅,Semantica is resolving,⚠️ conflicts,ConflictDetector,-,0.00s
✅,Semantica is reasoning,🤔 reasoning,RuleManager,-,0.01s
✅,Semantica is reasoning,🤔 reasoning,InferenceEngine,-,0.00s
✅,Semantica is reasoning,🤔 reasoning,ExplanationGenerator,-,0.01s


## Step 2: Define Rules


In [5]:
# Initialize Inference Engine
engine = InferenceEngine()

# Define rules using logic syntax
rules = [
    "IF parent_of(?a, ?b) AND parent_of(?b, ?c) THEN grandparent_of(?a, ?c)",
    "IF lives_in(?x, ?y) AND located_in(?y, ?z) THEN lives_in(?x, ?z)"
]

for rule in rules:
    engine.add_rule(rule)


## Step 3: Forward Chaining


In [6]:
# Load facts from relationships into the engine
for rel in relationships:
    # Format: predicate(subject, object)
    fact_str = f"{rel['type']}({rel['source']}, {rel['target']})"
    engine.add_fact(fact_str)

# Perform forward chaining to derive new facts
results = engine.forward_chain()

print(f"Inferred {len(results)} new facts:")
for result in results:
    print(f" - {result.conclusion} (Rule: {result.rule_used.name})")


Inferred 2 new facts:
 - grandparent_of(alice, charlie) (Rule: Rule 1)
 - lives_in(alice, california) (Rule: Rule 2)


## Step 4: Backward Chaining


In [7]:
# Define a goal to prove
goal = "grandparent_of(alice, charlie)"

# Perform backward chaining
proof = engine.backward_chain(goal)

if proof:
    print(f"Goal '{goal}' proven successfully!")
else:
    print(f"Could not prove goal '{goal}'.")


Goal 'grandparent_of(alice, charlie)' proven successfully!


## Step 5: Generate Explanations


In [8]:
generator = ExplanationGenerator()

# Explain the last forward chaining inference
if results:
    explanation = generator.generate_explanation(results[0])
    print("Explanation for first inferred fact:")
    print(explanation.natural_language)

# If we have a proof from backward chaining, explain it
if proof:
    proof_explanation = generator.generate_explanation(proof)
    print("\nExplanation for backward chaining proof:")
    print(proof_explanation.natural_language)


Explanation for first inferred fact:
Given the premises: parent_of(alice, bob), parent_of(bob, charlie), we conclude: grandparent_of(alice, charlie) using rule 'Rule 1'.

Explanation for backward chaining proof:
Given the premises: , we conclude: grandparent_of(alice, charlie).


## Summary

Reasoning and inference workflow:
- Knowledge Graph Built
- Inference Rules Defined
- Facts Loaded into Engine
- Forward Chaining Performed
- Backward Chaining Performed
- Explanations Generated


---

### Deep Dive: Reasoning Module

This section provides an in-depth guide to Semantica's reasoning capabilities. Learn rule syntax, fact formats, chaining strategies, and explanation generation with robust, reproducible examples.

**What you'll practice**
- Defining rules with variables and predicates
- Loading facts in predicate form
- Running forward and backward chaining
- Generating human-readable explanations


In [9]:
from semantica.kg import GraphBuilder
from semantica.reasoning import InferenceEngine, ExplanationGenerator

builder = GraphBuilder()
engine = InferenceEngine()
explainer = ExplanationGenerator()


### Rule Syntax

Rules use predicate logic with variables prefixed by `?`.

- Example: `IF parent_of(?a, ?b) AND parent_of(?b, ?c) THEN grandparent_of(?a, ?c)`
- Variables unify across predicates in the same rule
- Conclusions are added as new facts when conditions match


In [10]:
entities = [
    {"id": "alice", "type": "Person", "name": "Alice"},
    {"id": "bob", "type": "Person", "name": "Bob"},
    {"id": "charlie", "type": "Person", "name": "Charlie"},
    {"id": "sf", "type": "Location", "name": "San Francisco"},
    {"id": "california", "type": "Location", "name": "California"}
]

relationships = [
    {"source": "alice", "target": "bob", "type": "parent_of"},
    {"source": "bob", "target": "charlie", "type": "parent_of"},
    {"source": "sf", "target": "california", "type": "located_in"},
    {"source": "alice", "target": "sf", "type": "lives_in"}
]

knowledge_graph = builder.build([{"entities": entities, "relationships": relationships}])
print(len(knowledge_graph.get("entities", [])))
print(len(knowledge_graph.get("relationships", [])))


4
4


In [11]:
rules = [
    "IF parent_of(?a, ?b) AND parent_of(?b, ?c) THEN grandparent_of(?a, ?c)",
    "IF lives_in(?x, ?y) AND located_in(?y, ?z) THEN lives_in(?x, ?z)"
]
for r in rules:
    engine.add_rule(r)


In [12]:
for rel in relationships:
    fact = f"{rel['type']}({rel['source']}, {rel['target']})"
    engine.add_fact(fact)

derived = engine.forward_chain()
print(len(derived))
for d in derived:
    print(d.conclusion)


2
grandparent_of(alice, charlie)
lives_in(alice, california)


In [13]:
goals = [
    "grandparent_of(alice, charlie)",
    "lives_in(alice, california)"
]
for g in goals:
    proof = engine.backward_chain(g)
    print(g)
    print(bool(proof))


grandparent_of(alice, charlie)
True
lives_in(alice, california)
True


In [14]:
if derived:
    exp = explainer.generate_explanation(derived[0])
    print(exp.natural_language)

goal = "grandparent_of(alice, charlie)"
proof = engine.backward_chain(goal)
if proof:
    pexp = explainer.generate_explanation(proof)
    print(pexp.natural_language)


Given the premises: parent_of(alice, bob), parent_of(bob, charlie), we conclude: grandparent_of(alice, charlie) using rule 'Rule 1'.
Given the premises: , we conclude: grandparent_of(alice, charlie).
